In [1]:
from google.colab import drive
mydrive = '/content/drive/'
drive.mount(mydrive)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
import cv2, glob, os
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from keras.preprocessing.image import img_to_array, load_img

Using TensorFlow backend.


In [0]:
def loading_data(path):
    print('Loading data...')
    imgs = []
    img_path = os.path.join(path , '*.jpg')
    imgs = imgs + [img_to_array(load_img(img)) for img in glob.glob(img_path)]
        # target_size remains same
          
    print('Total image number:' , len(imgs))
    print('Loading Done.')
    return imgs

In [0]:
def convert_data(images):
  pixel_values = []
  image_shapes = []
  for image in images:
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_shapes.append(image.shape)
    # reshape the image to a 2D array of pixels and 3 color values (RGB)
    # convert to float ( cv2.kmeans() expects float)
    pixel_values.append( np.float32(image.reshape((-1, 3))) ) 
  return pixel_values, image_shapes

In [0]:
def k_mean_process(pixel_values,k,criteria):

  print('Segmentation start.')
  labels_list = []
  centers_list = []
  for  i in range(len(pixel_values)):
    print('Computing image '+ str(i) +'...')
    _, labels, (centers) = cv2.kmeans(pixel_values[i], k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    # convert back to 8 bit values
    centers = np.uint8(centers)
    # flatten the labels array
    labels = labels.flatten()

    labels_list.append(labels)
    centers_list.append(centers)  
  print('Segmentation done.')
  return labels_list, centers_list

In [0]:
def segmenting(labels_list, centers_list, image_shapes):
  segmented_images = []
  for i in range(len(labels_list)):
    # convert all pixels to the color of the centroids
    segmented_image = centers_list[i][labels_list[i].flatten()]
    # reshape back to the original image dimension
    segmented_image = segmented_image.reshape(image_shapes[i])

    segmented_images.append(segmented_image)
  return segmented_images

In [0]:
def kmeans_image_save(directory, image_names, images):
  for i in range(len(images)):
    name = os.path.splitext(image_names[i])[:-1]
    separator = ''
    save_as = os.path.join(directory, (separator.join(name) +'_kmeans.jpg') )
    cv2.imwrite(save_as,images[i])
  print('ALL '+str(len(images))+' images are saved.' )

In [8]:
src_path = '/content/drive/My Drive/Project/0425guava-classified'

proc_path = '/content/drive/My Drive/Project/K_dataset'

for subpath in os.listdir(src_path):
  data_path = os.path.join(src_path, subpath)
  images = loading_data(data_path)
  

  pixel_values, image_shapes = convert_data(images)
  # define stopping criteria
  criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
  # k: number of clusters
  k = 5
  labels_list, centers_list = k_mean_process(pixel_values,k,criteria)
  segmented_images = segmenting(labels_list, centers_list, image_shapes)

  print('Saving...')
  image_names = os.listdir(data_path)
  directory = os.path.join(proc_path, subpath)
  if not os.path.exists(directory):
    os.makedirs(directory)
  kmeans_image_save(directory, image_names, segmented_images)
print('Done.')

Loading data...
Total image number: 270
Loading Done.
Segmentation start.
Computing image 0...
Computing image 1...
Computing image 2...
Computing image 3...
Computing image 4...
Computing image 5...
Computing image 6...
Computing image 7...
Computing image 8...
Computing image 9...
Computing image 10...
Computing image 11...
Computing image 12...
Computing image 13...
Computing image 14...
Computing image 15...
Computing image 16...
Computing image 17...
Computing image 18...
Computing image 19...
Computing image 20...
Computing image 21...
Computing image 22...
Computing image 23...
Computing image 24...
Computing image 25...
Computing image 26...
Computing image 27...
Computing image 28...
Computing image 29...
Computing image 30...
Computing image 31...
Computing image 32...
Computing image 33...
Computing image 34...
Computing image 35...
Computing image 36...
Computing image 37...
Computing image 38...
Computing image 39...
Computing image 40...
Computing image 41...
Computing im

KeyboardInterrupt: ignored

========================================================

In [0]:
src_path = '/content/drive/My Drive/Project/0425guava-classified'

proc_path = '/content/drive/My Drive/Project/K_dataset'

for subpath in os.listdir(src_path):
  data_path = os.path.join(src_path, subpath)
  images = loading_data(data_path)
  

  pixel_values, image_shapes = convert_data(images)
  # define stopping criteria
  criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
  # k: number of clusters
  k = 5
  labels_list, centers_list = k_mean_process(pixel_values,k,criteria)
  segmented_images = segmenting(labels_list, centers_list, image_shapes)

  print('Saving...')
  image_names = os.listdir(data_path)
  directory = os.path.join(proc_path, subpath)
  if not os.path.exists(directory):
    os.makedirs(directory)
  kmeans_image_save(directory, image_names, segmented_images)
print('Done.')

In [0]:
print('Total image number:' , len(images))
for i in range(len(images)):

  plt.subplot(1,2,1)
  plt.imshow((images[i]).astype(np.uint8))
  plt.title(str(i)+'-before')
  plt.subplot(1,2,2)
  plt.imshow(cv2.cvtColor(segmented_images[i], cv2.COLOR_BGR2RGB))
  plt.title(str(i)+'-after')
  plt.show()

# ======================================

In [0]:
subpath = os.listdir('/content/drive/My Drive/Project/0425guava')
subpath
data_path = os.path.join(src_path , subpath[0])
data_path
cwd = os.getcwd()
cwd